# 手書き文字認識ハンズオン

In [0]:
# ライブラリの読み込み
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# データ読み込み
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# one-hot-label に変換
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# モデルの構築
model = Sequential()

# 最初の隠れ層(入力784 -> 出力256)
model.add(Dense(256, activation="sigmoid",input_shape=(784,)))
model.add(Dropout(0.1))

# 2つ目の隠れ層(入力256 -> 出力512)
## 隠れ層は以下の2行をコピペすることで追加できます
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.5))

# 3つ目の隠れ層(入力512 -> 出力100)
## 隠れ層は以下の2行をコピペすることで追加できます
model.add(Dense(100, activation='sigmoid'))
model.add(Dropout(0.1))

# 出力層(入力100 -> 出力10)
model.add(Dense(10, activation='softmax'))

# モデル概要表示
model.summary()

In [0]:
# モデルのコンパイル
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

In [0]:
# バッチサイズ
batch_size = 100

# エポック数
epochs = 10

# フィッティング(学習)
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

In [0]:
# スコアの計算・表示
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# モデルの評価
model.evaluate(x_test, y_test, verbose=1)

In [0]:
# 学習の様子をプロット
loss = history.history['loss']
val_loss = history.history['val_loss']

acc = history.history['acc']
val_acc = history.history['val_acc']

plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.title('Loss')
epochs = len(loss)
plt.plot(range(epochs), loss, marker='.', label='loss')
plt.plot(range(epochs), val_loss, marker='.', label='val_loss')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')

plt.subplot(2,1,2)
plt.title('Accuracy')
plt.plot(range(epochs), acc, marker='.', label='acc')
plt.plot(range(epochs), val_acc, marker='.', label='val_acc')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show()